In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125005
Azure region: southcentralus
Subscription id: de8aba62-c352-42be-b980-2faedf08ead8
Resource group: aml-quickstarts-125005


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, loguniform, choice
import shutil
import os

# Specify parameter sampler

ps = RandomParameterSampling( {
        "--C" : choice(1,2,4),
        "--max_iter" : choice(10,50,100)
      
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', "./training")
    
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training', 
              entry_script = 'train.py', 
              compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     policy = policy,
                                     estimator = est)

In [24]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_483ea498-b498-4279-8ad3-7a3dd612316e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_483ea498-b498-4279-8ad3-7a3dd612316e?wsid=/subscriptions/de8aba62-c352-42be-b980-2faedf08ead8/resourcegroups/aml-quickstarts-125005/workspaces/quick-starts-ws-125005

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-07T10:23:20.923294][API][INFO]Experiment created<END>\n""<START>[2020-11-07T10:23:21.471961][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-07T10:23:21.640821][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-07T10:23:23.0240644Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()



In [ ]:
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
best_run_metrics = best_run.get_metrics()

In [ ]:

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

model = best_run.register_model(model_name='model_project1', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80,test_size=0.20, random_state=1)
x_df = x_train.assign(y_train=y_train.values)
df = pd.DataFrame(x_df)
df.to_csv("training/train_data.csv", index=False)


ds = ws.get_default_datastore()
ds.upload(src_dir='./training', target_path='bankmarketing', overwrite=True, show_progress=True)

train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5)

In [ ]:
# Submit your automl run
from azureml.widgets import RunDetails
remote_run = exp.submit(automl_config, show_output = True)
RunDetails(automl_run).show()
remote_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_run_customized, fitted_model_customized = remote_run.get_output()

In [ ]:
best_automl_run.register_model(model_name = "automlproject.pkl", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
compute_target.delete()